# Combine Sports Reference and Kenpom data into one dataframe

In [8]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# head_names = ['School', 'W-L%', 'SRS', 'SOS', 'TmPts', 'OppPts', 
#               'Pace', 'ORtg', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 
#               'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA']

kpHeadNames=['Rank','School','Conf','WL','AdjEM','AdjO','AdjD','AdjT','Luck','AdjEM','OppO','OppD','NCSOS_AdjEM','junk']

def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)


years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']


In [9]:
kpData=pd.read_csv('data/kp/kp-2017.csv',names=kpHeadNames).drop('junk',axis=1)
teamNamesKP=kpData.School.values

newTeamList=[]
for t in teamNamesKP:
    newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
teamNamesKP=newTeamList

In [10]:
srData.School.values

array(['abilene-christian', 'air-force', 'akron', 'alabama-a&m',
       'alabama-birmingham', 'alabama-state', 'alabama', 'albany-(ny)',
       'alcorn-state', 'american', 'appalachian-state', 'arizona-state',
       'arizona', 'arkansas-little-rock', 'arkansas-pine-bluff',
       'arkansas-state', 'arkansas', 'army', 'auburn', 'austin-peay',
       'ball-state', 'baylor', 'belmont', 'bethune-cookman', 'binghamton',
       'boise-state', 'boston-college', 'boston-university',
       'bowling-green-state', 'bradley', 'brigham-young', 'brown',
       'bryant', 'bucknell', 'buffalo', 'butler', 'cal-poly',
       'cal-state-bakersfield', 'cal-state-fullerton',
       'cal-state-northridge', 'uc-davis', 'uc-irvine', 'uc-riverside',
       'uc-santa-barbara', 'university-of-california', 'campbell',
       'canisius', 'central-arkansas', 'central-connecticut-state',
       'central-florida', 'central-michigan', 'charleston-southern',
       'charlotte', 'chattanooga', 'chicago-state', 'cincin

In [11]:
srData=pd.read_csv('data/team/team_stats_2017.csv')
teamNamesBR=srData.School

Matchup all names for consistency

In [12]:
print(len(teamNamesBR))
print(len(teamNamesKP))

351
351


In [13]:
teamNameMismatch=[]
for teamName in teamNamesKP:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

45

In [14]:
teamNamesKP

['gonzaga',
 'villanova',
 'north-carolina',
 'kentucky',
 'florida',
 'kansas',
 'west-virginia',
 'wichita-state',
 'louisville',
 'oregon',
 'smu',
 'virginia',
 'baylor',
 'duke',
 "saint-mary's",
 'ucla',
 'iowa-state',
 'arizona',
 'purdue',
 'michigan',
 'wisconsin',
 'oklahoma-state',
 'cincinnati',
 'south-carolina',
 'butler',
 'florida-state',
 'notre-dame',
 'creighton',
 'tcu',
 'kansas-state',
 'xavier',
 'marquette',
 'vanderbilt',
 'rhode-island',
 'arkansas',
 'wake-forest',
 'minnesota',
 'northwestern',
 'dayton',
 'michigan-state',
 'texas-tech',
 'miami-fl',
 'middle-tennessee',
 'indiana',
 'clemson',
 'maryland',
 'utah',
 'vcu',
 'illinois-state',
 'virginia-tech',
 'seton-hall',
 'houston',
 'usc',
 'nevada',
 'syracuse',
 'alabama',
 'tennessee',
 'princeton',
 'unc-wilmington',
 'providence',
 'georgia',
 'california',
 'vermont',
 'texas-a&m',
 'oklahoma',
 'illinois',
 'east-tennessee-state',
 'ucf',
 'georgetown',
 'texas',
 'iowa',
 'colorado',
 'ohio-sta

In [15]:
teamNameConversionKP2BR={
'smu': 'southern-methodist',
 "saint-mary's": "saint-mary's-(ca)",
 'tcu': 'texas-christian',
 'miami-fl': 'miami-(fl)',
 'vcu': 'virginia-commonwealth',
 'usc': 'southern-california',
 'unc-wilmington': 'north-carolina-wilmington',
 'california': 'university-of-california',
 'ucf': 'central-florida',
 'ut-arlington': 'texas-arlington',
 'byu': 'brigham-young',
 'state-bonaventure': 'st.-bonaventure',
 'loyola-chicago': 'loyola-(il)',
 "state-john's": "st.-john's-(ny)",
 'unc-asheville': 'north-carolina-asheville',
 'unc-greensboro': 'north-carolina-greensboro',
 'albany': 'albany-(ny)',
 'fort-wayne': 'ipfw',
 'penn': 'pennsylvania',
 'lsu': 'louisiana-state',
 'texas-a&m-corpus-chris': 'texas-a&m-corpus-christi',
 'uab': 'alabama-birmingham',
 'utep': 'texas-el-paso',
 'umbc': 'maryland-baltimore-county',
 "mount-state-mary's": "mount-st.-mary's",
 'umkc': 'missouri-kansas-city',
 'loyola-md': 'loyola-(md)',
 'usc-upstate': 'south-carolina-upstate',
 'little-rock': 'arkansas-little-rock',
 'unlv': 'nevada-las-vegas',
 'bowling-green': 'bowling-green-state',
 'utsa': 'texas-san-antonio',
 'liu-brooklyn': 'long-island-university',
 'state-francis-pa': 'saint-francis-(pa)',
 'fiu': 'florida-international',
 'miami-oh': 'miami-(oh)',
 'the-citadel': 'citadel',
 'umass-lowell': 'massachusetts-lowell',
 'ut-rio-grande-valley': 'texas-rio-grande-valley',
 'detroit': 'detroit-mercy',
 'prairie-view-a&m': 'prairie-view',
 'southern-miss': 'southern-mississippi',
 'grambling-state': 'grambling',
 'central-connecticut': 'central-connecticut-state',
 'state-francis-ny': 'st.-francis-(ny)'
}

In [16]:
teamNamesKPConverted=[]
for teamName in teamNamesKP:
    if teamName in teamNameConversionKP2BR.keys():
        teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
    else:
        teamNamesKPConverted.append(teamName)
len(teamNamesKPConverted)

351

Remaining number of mismatches

In [17]:
teamNameMismatch=[]
for teamName in teamNamesKPConverted:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

0

In [18]:

teamNameMismatch

[]

In [19]:
tquery='california-davis'
for teamName in teamNamesBR:
    if tquery in teamName:
        print(tquery, ' is in')
        

# Combine dataframes

In [20]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [21]:
kpData.School.values

array(['Gonzaga', 'Villanova', 'North Carolina', 'Kentucky', 'Florida',
       'Kansas', 'West Virginia', 'Wichita St.', 'Louisville', 'Oregon',
       'SMU', 'Virginia', 'Baylor', 'Duke', "Saint Mary's", 'UCLA',
       'Iowa St.', 'Arizona', 'Purdue', 'Michigan', 'Wisconsin',
       'Oklahoma St.', 'Cincinnati', 'South Carolina', 'Butler',
       'Florida St.', 'Notre Dame', 'Creighton', 'TCU', 'Kansas St.',
       'Xavier', 'Marquette', 'Vanderbilt', 'Rhode Island', 'Arkansas',
       'Wake Forest', 'Minnesota', 'Northwestern', 'Dayton',
       'Michigan St.', 'Texas Tech', 'Miami FL', 'Middle Tennessee',
       'Indiana', 'Clemson', 'Maryland', 'Utah', 'VCU', 'Illinois St.',
       'Virginia Tech', 'Seton Hall', 'Houston', 'USC', 'Nevada',
       'Syracuse', 'Alabama', 'Tennessee', 'Princeton', 'UNC Wilmington',
       'Providence', 'Georgia', 'California', 'Vermont', 'Texas A&M',
       'Oklahoma', 'Illinois', 'East Tennessee St.', 'UCF', 'Georgetown',
       'Texas', 'Iowa', 'Colo

In [22]:
for yr in np.arange(2017,2001,-1):
    # pull kenpom data
    print(yr)
    kpData=pd.read_csv('data/kp/kp-'+str(yr)+'.csv',names=kpHeadNames).drop('junk',axis=1)
    teamNamesKP=kpData.School.values

    #basic formatting of school names
    newTeamList=[]
    for t in teamNamesKP:
        newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
    teamNamesKP=newTeamList

    # convert remaining names
    teamNamesKPConverted=[]
    for teamName in teamNamesKP:
        if teamName in teamNameConversionKP2BR.keys():
            teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
        else:
            teamNamesKPConverted.append(teamName)

    kpData.School=teamNamesKPConverted
    
    

    srData=pd.read_csv('data/team/team_stats_'+str(yr)+'.csv')
    teamNamesBR=srData.School

    allData=srData.merge(kpData,how='inner',on='School')
    allData=allData.rename(columns={'W%': 'wpct'})
    
    allData.to_csv("data/combined_team_stats_" + str(yr) + ".csv")
    
#     break
    # allData.sort_values(by='W-L%',ascending=False)
#     wpct=[]
#     for r in allData.itertuples():
#         w,l = r.WL.split('-')
#         wpct.append(float(w)/(float(w)+float(l)))
#     allData['wpct2']=wpct
#     #     print()
#     plt.figure()
#     plt.title(yr)
#     allData[['wpct','wpct2']]
#     plt.hist((allData['wpct']-allData['wpct2']));

2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
